# Controlled-Z Gate

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
In this tutorial, we will implement a controlled-Z gate using Quanlse Cloud Service. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Construct Hamiltonian
- Generate and optimize pulse via Quanlse Cloud Service
- Summary

## Introduction

Theoretically, the controlled-Z gate (CZ gate) is a two-qubit quantum gate that adds a phase of $e^{i\pi}$ to the target qubit if the control qubit and target qubit's state is $|11\rangle$. The physical implementation of the CZ gate is realized using magnetic flux to tune the qubit's eigenfrequency - this is accomplished by slowly increasing the magnetic flux, waiting for some time to add a phase $\pi$ and then decreasing the flux back to 0 \[1\]. CZ gate allows for faster two-qubit control than the cross-resonance gate due to stronger inductive coupling. 

The matrix representation of $U_{\rm CZ}$ is:
$$
U_{\rm CZ} = |0\rangle\langle 0| \otimes I + |1\rangle\langle1| \otimes \hat{\sigma}^z = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix}. 
$$  

Using the circuit identity, the CNOT gate can be implemented by one CZ gate and two Hadamard gates since $H\hat{\sigma}^zH=\hat{\sigma}^x$ \[1\]. 

![cnot](figures/cz-cnot.png)

## Preparation
After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import Hamiltonian-related modules 
from Quanlse.QHamiltonian import QHamiltonian as QHam
from Quanlse.QOperator import duff, number

# Import the optimizer for controlled-z gate
from Quanlse.remoteOptimizer import remoteOptimizeCz

# Import tools for result analysis
from Quanlse.Utils.Functions import project

# Import numpy and math
from numpy import round
from math import pi

## Construct  Hamiltonian

In our modeling, we will account for energy leakage for a two-qubit system by including the third energy level. The system Hamiltonian we will define is:

$$
\hat{H}_{\rm sys}(t) = (\omega_{\rm q0}-\omega_{\rm d0})\hat{a}_0^\dagger \hat{a}_0+(\omega_{\rm q1}-\omega_{\rm d0}) \hat{a}_1^\dagger \hat{a}_1+\frac{\alpha_0}{2}\hat{a}_0^{\dagger}\hat{a}_0^\dagger\hat{a}_0\hat{a}_0 + \frac{\alpha_1}{2}\hat{a}_1^\dagger\hat{a}_1^\dagger\hat{a}_1\hat{a}_1 + \frac{g}{2}(\hat{a}_0\hat{a}_1^\dagger+\hat{a}_0^\dagger\hat{a}_1) + \frac{A_0^z(t)}{2}\hat{a}_0^\dagger \hat{a}_0,
$$

where $\hat{a}_i^\dagger$, $\hat{a}_i$ are the creation and annihilation operators for the qubit $q_i$ ($i$=0, 1). The information regarding the hardware structure is specified by parameters: qubit frequency $\omega_{qi}$, drive frequency $\omega_{{\rm d}i}$, anharmonicity $\alpha_i$, and coupling strength $g$. $A_0^z(t)$ is the magnetic flux applying on qubit $q_0$.

In Quanlse, we will construct the system Hamiltonian by adding up the three terms:
$$
\hat{H}_{\rm sys}(t) = \hat{H}_{\rm drift} + \hat{H}_{\rm coup} + \hat{H}_{\rm ctrl}(t). 
$$

We will first define the necessary function arguments, including the sampling period, number of qubits, and the system's energy levels to consider. We then use the function ` QHamiltonian()` to initialize the Hamiltonian dictionary by passing in the variables we just defined.

In [ ]:
# Sampling period
dt = 1.0

# Number of qubits
qubits = 2

# System energy levels
level = 3

# Initilize the Hamiltonian
ham = QHam(subSysNum=qubits, sysLevel=level, dt=dt)

After initializing the Hamiltonian dictionary, we will start to add terms to the Hamiltonian. Here, we define the parameters of the hardware that would be used later:

In [ ]:
# Specify the parameters of the hardware
qubitArgs = {
    "coupling": 0.0277 * (2 * pi),  # Coupling of Q0 and Q1
    "qubit_freq0": 5.805 * (2 * pi),  # Frequency of Q0
    "qubit_freq1": 5.205 * (2 * pi),  # Frequency of Q1
    "drive_freq0": 5.205 * (2 * pi),  # Drive frequency on Q0 (rotating frame)
    "drive_freq1": 5.205 * (2 * pi),  # Drive frequency on Q1 (rotating frame)
    "qubit_anharm0": -0.217 * (2 * pi),  # Anharmonicity of Q0
    "qubit_anharm1": -0.226 * (2 * pi)  # Anharmonicity of Q1
}

Next, we add the drift terms to the initalized Hamiltonian. The drift Hamiltonian $\hat{H}_{\rm drift}$ takes the form:
$$
\hat{H}_{\rm drift} = (\omega_{\rm q0}-\omega_{\rm d0})\hat{n}_0+(\omega_{\rm q1}-\omega_{\rm d0})\hat{n}_1+\frac{\alpha_0}{2}\hat{a}_0^\dagger\hat{a}_0^\dagger\hat{a}_0\hat{a}_0+\frac{\alpha_1}{2}\hat{a}_1^\dagger\hat{a}_1^\dagger\hat{a}_1\hat{a}_1 .
$$
Here, $\hat{n}_i=\hat{a}^\dagger_i \hat{a}_i$ is the number operator for qubit $q_i$. In Quanlse, this can be done using the function `addDrift()` which takes a Hamiltonian dictionary, a user-defined name, the list of qubit(s) the term acts upon, the according operators, and the amplitude:

In [ ]:
# Add drift term(s)
for qu in range(2):
    # Add detuning term(s)
    ham.addDrift(number, qu, (qubitArgs[f"qubit_freq{qu}"] - qubitArgs[f"drive_freq{qu}"]))
    # Add anharmonicity term(s)
    ham.addDrift(duff, qu, qubitArgs[f"qubit_anharm{qu}"] / 2)

Now, we add the coupling Hamiltonian $\hat{H}_{\rm coup}$, which takes the form:
$$
\hat{H}_{\rm coup} = \frac{g}{2}(\hat{a}_0\hat{a}_1^\dagger+\hat{a}_0^\dagger\hat{a}_1). 
$$

The coupling Hamiltonian is added by the function `addCoupling()`. The function arguments include a Hamiltonian dictionary, a user-defined name, a list of qubit indices which the term acts upon, and the coupling strength. 

In [ ]:
# Add coupling term
ham.addCoupling([0, 1], qubitArgs["coupling"] / 2)

Noted that the optimizer in Quanlse will automatically add the control term:

$$ 
\hat{H}_{\rm ctrl}(t) = \frac{A_0^z(t)}{2}\hat{a}_0^\dagger \hat{a}_0, 
$$

thus we don't need to add this term manually.

With the system Hamiltonian built, we can now move on to the optimization.

## Generate and optimize pulse via Quanlse Cloud Service

The optimization process usually takes a long time to process on local devices, however, we provide a cloud service that could speed up this process significantly. To use the Quanlse Cloud Service, the users need to acquire a token from http://quantum-hub.baidu.com.

In [ ]:
# Import tools to get access to cloud service
from Quanlse import Define

# To use remoteOptimizerCz() on cloud, paste your token (a string) here
Define.hubToken = ''

For this example, we can use `remoteOptimizeCz()` to generate and optimize the control pulses at the designated `targetInfidelity`. To use this function, the user needs to specify a Hamiltonian dictionary, the amplitude's bound, gate time, maximum iteration number, and the target infidelity. It returns a Hamiltonian and the infidelity found at the local minimum. `aBound` sets the bound for the strength of our pulse - a larger bound will lead to a larger search space. Hence, we can increase the number of iterations to reach a better result.

In [ ]:
aBound=(-5, -1)  # The bound of the pulse's strength 
gateJob, infidelity = remoteOptimizeCz(ham, aBound=aBound, tg=40, maxIter=5, targetInfidelity=0.005)

We now extract the optimized pulse and the infidelity. The gate infidelity for performance assessment throughout this tutorial is defined as ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}[U^\dagger_{\rm goal}P(U)]\right|$, where $U_{\rm goal}=U_{\rm CZ}$, $d$ is the dimension of $U_{\rm goal}$ and $U$ is the unitary evolution of the system defined previously. The projected evolution $P(U)$ in particular describes the evolution projected to the computational subspace.

In [ ]:
print(f"minimum infidelity: {infidelity}")
gateJob.plot()

The system we defined previously is constituted by two three-level qubits. This indicates that the time evolution operator for this system is a $9\times 9$ matrix. To simulate the evolution, you can use `simulate()`, The projected evolution matrix $P(U)$ can be obtained by `project()`:

In [ ]:
result = ham.simulate(job=gateJob)
process2d = project(result.result[0]["unitary"], qubits, level, 2)
print("The projected evolution P(U):\n", round(process2d, 2))

## Summary

This tutorial introduces controlled-Z gate's implementation using Quanlse Cloud Service. The users could follow this link [tutorial-cz-gate.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-cz.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.

## References
\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)